# Data Extraction

In [2]:
import pandas as pd
import seaborn as sns
import requests

In [3]:
url = 'https://www.the-numbers.com/movie/budgets/all'
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'}
html = requests.get(url, headers=headers).content
df_list = pd.read_html(html)
df = df_list[-1]
df.head()

,Unnamed: 0,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross
0,1,"Dec 9, 2022",Avatar: The Way of Water,"$460,000,000","$657,411,991","$2,243,611,991"
1,2,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,794,731,755"
2,3,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,071,802","$1,045,713,802"
3,4,"Apr 22, 2015",Avengers: Age of Ultron,"$365,000,000","$459,005,868","$1,395,316,979"
4,5,"May 17, 2023",Fast X,"$340,000,000",$0,$0
